In [1]:
import os
import gc
import datetime
import warnings
import time

import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import LinearRegression, HuberRegressor, ElasticNet
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA

os.chdir(r'/content/drive/MyDrive/Replicate/Gu et al.(2020)')


warnings.filterwarnings('ignore')

%matplotlib inline

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: '/content/drive/MyDrive/Replicate/Gu et al.(2020)'

In [ ]:
def data_preprocessing(data_ch, data_ma, stdt, nddt):
    ## firm characteristics
    data_ch = data_ch[(data_ch['DATE']>=stdt)&(data_ch['DATE']<=nddt)].reset_index(drop=True)
    data_ch['DATE'] = pd.to_datetime(data_ch['DATE'],format='%Y%m%d')+pd.offsets.MonthEnd(0)
    characteristics = list(set(data_ch.columns).difference({'permno','DATE','SHROUT','mve0','sic2','RET','prc'}))
    data_ch = fill_na(data_ch, characteristics)
    data_ch = XS_MinMax(data_ch, characteristics)
    data_ch = get_sic_dummies(data_ch)

    ## macroeconomic predictors
    data_ma = data_ma[(data_ma['yyyymm']>=stdt//100)&(data_ma['yyyymm']<=nddt//100)].reset_index(drop=True)
    ma_predictors = ['dp_sp','ep_sp','bm_sp','ntis','tbl','tms','dfy','svar']
    data_ma['Index'] = data_ma['Index'].str.replace(',','').astype('float64')
    data_ma['dp_sp'] = np.log(data_ma['D12'])-np.log(data_ma['Index'])
    data_ma['ep_sp'] = np.log(data_ma['E12'])-np.log(data_ma['Index'])
    data_ma.rename({'b/m':'bm_sp'},axis=1,inplace=True)
    data_ma['tms'] = data_ma['lty']-data_ma['tbl']
    data_ma['dfy'] = data_ma['BAA']-data_ma['AAA']
    data_ma = data_ma[['yyyymm']+ma_predictors]
    data_ma['yyyymm'] = pd.to_datetime(data_ma['yyyymm'],format='%Y%m')+pd.offsets.MonthEnd(0)

    ## merge and add interaction variables
    data = interactions(data_ch, data_ma, characteristics, ma_predictors)
    data = data.set_index('DATE')
    data = data.sort_values(['DATE','permno'])
    return data

In [ ]:
# filling missing data
def fill_na(data_ch, characteristics):
    for ch in characteristics:
         data_ch[ch] = data_ch.groupby('DATE')[ch].transform(lambda x: x.fillna(x.median()))
    for ch in characteristics:
         data_ch[ch] = data_ch[ch].fillna(0)
    return data_ch

In [ ]:
# cross-sectional transformation to [-1,1]
def XS_MinMax(data_ch, characteristics):
    for ch in characteristics:
         data_ch[ch] = data_ch.groupby('DATE')[ch].transform(lambda x: minmax_scale(np.array(x).reshape((len(x),1)),(-1,1)).flatten())
    return data_ch

In [ ]:
# get dummy variables for sic code
def get_sic_dummies(data_ch):
    sic_dummies = pd.get_dummies(data_ch['sic2'].fillna(999).astype(int),prefix='sic').drop('sic_999',axis=1)
    data_ch_d = pd.concat([data_ch,sic_dummies],axis=1)
    data_ch_d.drop(['prc','SHROUT','mve0','sic2'],inplace=True,axis=1)
    return data_ch_d

In [ ]:
# merge and add interaction variables
def interactions(data_ch, data_ma, characteristics, ma_predictors):
    # construct interactions between firm characteristics and macroeconomic predictors
    data = data_ch.copy()
    data_ma_long = pd.merge(data[['DATE']],data_ma,left_on='DATE',right_on='yyyymm',how='left')
    data = data.reset_index(drop=True)
    data_ma_long = data_ma_long.reset_index(drop=True)
    for fc in characteristics:
        for mp in ma_predictors:
            data[fc+'*'+mp] = data[fc]*data_ma_long[mp]
    return data

In [ ]:
## validation function to find the best model
def val_fun(model, params: dict, X_trn, y_trn, X_vld, y_vld):
    best_ros = None
    lst_params = list(ParameterGrid(params))
    for param in lst_params:
        if best_ros == None:
            mod = model().set_params(**param).fit(X_trn, y_trn)
            best_mod = mod
            y_pred = mod.predict(X_vld)
            best_ros = R_oos(y_vld, y_pred)
            best_param = param
        else:
            mod = model().set_params(**param).fit(X_trn, y_trn)
            y_pred = mod.predict(X_vld)
            ros = R_oos(y_vld, y_pred)
            if ros > best_ros:
                best_ros = ros
                best_mod = mod
                best_param = param
    return best_mod

## Metric

In [ ]:
def R_oos(actual, predicted):
    actual, predicted = np.array(actual).flatten(), np.array(predicted).flatten()
    #predicted = np.where(predicted<0,0,predicted)
    return 1 - (np.dot((actual-predicted),(actual-predicted)))/(np.dot(actual,actual))

## Save

In [ ]:
def save_pred(file_path,res,res_top,res_bot):
    if not os.path.exists(file_path+'prediction'):
        os.mkdir(file_path+'prediction')
    if not os.path.exists(file_path+'results'):
        os.mkdir(file_path+'results')

    res.to_csv(file_path+'prediction/res.csv')
    res_top.to_csv(file_path+'prediction/res_top.csv')
    res_bot.to_csv(file_path+'prediction/res_bot.csv')

## Linear Model

In [ ]:
## OLS with expanding window
def EXP_OLS(data,fts,tgt,yrs_trn):
    X = data[fts]
    y = data[[tgt]]
    y_pred = []
    R2_oos = []

    # deal with dates
    all_dates = pd.Series(data.index).drop_duplicates().sort_values().reset_index(drop=True)

    # train test split and prediction
    for i in np.arange(12*yrs_trn-1,len(all_dates[:-12]),12):
        nddt_trn = all_dates[i]
        nddt_tst = all_dates[i+12]
        X_trn, y_trn = X[X.index<=nddt_trn], y[y.index<=nddt_trn]
        X_tst, y_tst= X[(X.index>nddt_trn)&(X.index<=nddt_tst)], y[(y.index>nddt_trn)&(y.index<=nddt_tst)]
        mod = LinearRegression().fit(X_trn,y_trn)
        y_pred = list(mod.predict(X_tst))
        R2_oos.append(R_oos(y_tst, mod.predict(X_tst)))

    return y_pred, R2_oos

In [ ]:
## Huber regression with expanding window
def EXP_HUBER(data,fts,tgt,yrs_trn,epsilon):
    X = data[fts]
    y = data[[tgt]]
    y_pred = []

    # deal with dates
    all_dates = pd.Series(data.index).drop_duplicates().sort_values().reset_index(drop=True)

    # train test split and prediction
    for i in np.arange(12*yrs_trn-1,len(all_dates[:-12]),12):
        nddt_trn = all_dates[i]
        nddt_tst = all_dates[i+12]
        X_trn, y_trn = X[X.index<=nddt_trn], y[y.index<=nddt_trn]
        X_tst, y_tst= X[(X.index>nddt_trn)&(X.index<=nddt_tst)], y[(y.index>nddt_trn)&(y.index<=nddt_tst)]
        mod = HuberRegressor(epsilon=epsilon).fit(X_trn,y_trn)
        y_pred += list(mod.predict(X_tst).flatten())

    R2_oos = R_oos(y_tst ,y_pred)
    return R2_oos

In [ ]:
## PLS with expanding window
def EXP_PLS(data,fts,tgt,yrs_trn,yrs_vld,params):
    X = data[fts]
    y = data[[tgt]]
    y_pred = []

    # deal with dates
    all_dates = pd.Series(data.index).drop_duplicates().sort_values().reset_index(drop=True)

    # train validate test split and prediction
    for i in np.arange(12*yrs_trn-1,len(all_dates[:-12*(yrs_vld+1)]),12):
        nddt_trn = all_dates[i]
        nddt_vld = all_dates[i+yrs_vld*12]
        nddt_tst = all_dates[i+(yrs_vld+1)*12]
        X_trn, y_trn = X[X.index<=nddt_trn], y[y.index<=nddt_trn]
        X_vld, y_vld = X[(X.index>nddt_trn)&(X.index<=nddt_vld)], y[(y.index>nddt_trn)&(y.index<=nddt_vld)]
        X_tst, y_tst = X[(X.index>nddt_vld)&(X.index<=nddt_tst)], y[(y.index>nddt_vld)&(y.index<=nddt_tst)]
        mod = val_fun(PLSRegression, params, X_trn, y_trn, X_vld, y_vld)
        y_pred += list(mod.predict(X_tst).flatten())

    R2_oos = R_oos(y_tst ,y_pred)
    return R2_oos

In [ ]:
## PCR class
class PCRegressor:

    def __init__(self,n_PCs=1,loss='mse'):
        self.n_PCs = n_PCs
        if loss not in ['huber','mse']:
            raise AttributeError(
            f"The loss should be either 'huber' or 'mse', but {loss} is given"
            )
        else:
            self.loss = loss

    def set_params(self, **params):
        for param in params.keys():
            setattr(self, param, params[param])
        return self

    def fit(self,X,y):
        X = np.array(X)
        N,K = X.shape
        y = np.array(y).flatten()
        self.mu = np.mean(X,axis=0).reshape((1,K))
        self.sigma = np.std(X,axis=0).reshape((1,K))
        self.sigma = np.where(self.sigma==0,1,self.sigma)
        X = (X-self.mu)/self.sigma
        pca = PCA()
        X = pca.fit_transform(X)[:,:self.n_PCs]
        self.Vk = pca.components_.T[:,:self.n_PCs]
        if self.loss == 'mse':
            self.model = LinearRegression().fit(X,y)
        else:
            self.model = HuberRegressor().fit(X,y)
        self.beta = self.Vk.T @ self.model.coef_
        return self

    def predict(self,X):
        X = np.array(X)
        X = (X-self.mu)/self.sigma
        pred = X @ self.Vk @ self.beta + self.model.intercept_
        return pred

In [ ]:
## PCR with expanding window
def EXP_PCR(data,fts,tgt,yrs_trn,yrs_vld,params):
    X = data[fts]
    y = data[[tgt]]
    y_pred = []

    # deal with dates
    all_dates = pd.Series(data.index).drop_duplicates().sort_values().reset_index(drop=True)

    # train validate test split and prediction
    for i in np.arange(12*yrs_trn-1,len(all_dates[:-12*(yrs_vld+1)]),12):
        nddt_trn = all_dates[i]
        nddt_vld = all_dates[i+yrs_vld*12]
        nddt_tst = all_dates[i+(yrs_vld+1)*12]
        X_trn, y_trn = X[X.index<=nddt_trn], y[y.index<=nddt_trn]
        X_vld, y_vld = X[(X.index>nddt_trn)&(X.index<=nddt_vld)], y[(y.index>nddt_trn)&(y.index<=nddt_vld)]
        X_tst = X[(X.index>nddt_vld)&(X.index<=nddt_tst)]
        mod = val_fun(PCRegressor, params, X_trn, y_trn, X_vld, y_vld)
        y_pred += list(mod.predict(X_tst).flatten())

    return y_pred

In [ ]:
## ENet with expanding window
def EXP_ENet(data,fts,tgt,yrs_trn,yrs_vld,params):
    X = data[fts]
    y = data[[tgt]]
    y_pred = []

    # deal with dates
    all_dates = pd.Series(data.index).drop_duplicates().sort_values().reset_index(drop=True)

    # train validate test split and prediction
    for i in np.arange(12*yrs_trn-1,len(all_dates[:-12*(yrs_vld+1)]),12):
        nddt_trn = all_dates[i]
        nddt_vld = all_dates[i+yrs_vld*12]
        nddt_tst = all_dates[i+(yrs_vld+1)*12]
        X_trn, y_trn = X[X.index<=nddt_trn], y[y.index<=nddt_trn]
        X_vld, y_vld = X[(X.index>nddt_trn)&(X.index<=nddt_vld)], y[(y.index>nddt_trn)&(y.index<=nddt_vld)]
        X_tst = X[(X.index>nddt_vld)&(X.index<=nddt_tst)]
        mod = val_fun(ElasticNet, params, X_trn, y_trn, X_vld, y_vld)
        y_pred += list(mod.predict(X_tst).flatten())

    return y_pred

## Basic Setup 

In [ ]:
print("Run")
stdt, nddt = 20010131, 20201231
yrs_trn, yrs_vld = 6, 4
stdt_tst = str(stdt + (yrs_trn+yrs_vld)*10000)
stdt_tst = np.datetime64(stdt_tst[:4]+'-'+stdt_tst[4:6]+'-'+stdt_tst[6:])

file_path = './data/'
filename_ch = 'GKX_20201231.csv' # filename of firm characteristics data
filename_ma = 'PredictorData2022_Monthly.csv' # filename of macroeconomic predictors data

## Data Preprocessing

In [2]:
st = time.time()
## firm characteristics data
data_ch = pd.read_csv(file_path+filename_ch)
# pick out top and bottom 1000 firms
data_ch_top = data_ch.sort_values('mvel1',ascending=False).groupby('DATE').head(1000).reset_index(drop=True)
data_ch_bot = data_ch.sort_values('mvel1',ascending=False).groupby('DATE').tail(1000).reset_index(drop=True)

## macroeconomic predictors data
data_ma = pd.read_csv(file_path+filename_ma)

## data preprocessing
data = data_preprocessing(data_ch,data_ma,stdt,nddt)
data_top = data_preprocessing(data_ch_top,data_ma,stdt,nddt)
data_bot = data_preprocessing(data_ch_bot,data_ma,stdt,nddt)
# features in each dataset
features = list(set(data.columns).difference({'permno','DATE','RET'}))
features_top = list(set(data_top.columns).difference({'permno','DATE','RET'}))
features_bot = list(set(data_bot.columns).difference({'permno','DATE','RET'}))
elapsed_time = time.time() - st
print(f'Data preprocessed!!!!!\n Execution time: {time.strftime("%H:%M:%S", time.gmtime(elapsed_time))}')

NameError: name 'file_path' is not defined

## Model fit

In [ ]:
## df to store true and pred RET
#res = data[['permno','RET']][data.index>=stdt_tst].copy()
res_top = data_top[['permno','RET']][data_top.index>=stdt_tst].copy()
#res_bot = data_bot[['permno','RET']][data_bot.index>=stdt_tst].copy()

In [ ]:
### Linear Models ###
## OLS
st = time.time()
#res['OLS_PRED'] = EXP_OLS(data,features,'RET',yrs_trn+yrs_vld)
res_top['OLS_PRED'], res_top_roos_OLS= EXP_OLS(data_top,features_top,'RET',yrs_trn+yrs_vld)
#res_bot['OLS_PRED'] = EXP_OLS(data_bot,features_bot,'RET',yrs_trn+yrs_vld)
#save_pred(file_path,res,res_top,res_bot)

In [3]:
from joblib import Parallel, delayed

def calculate_variable_importance(data, fts, tgt, yrs_trn, model):
    # Variable importance storage
    X = data[fts]
    y = data[[tgt]]
    R2_oos_all = []
    var_imp_dict = {}

    # deal with dates
    all_dates = pd.Series(data.index).drop_duplicates().sort_values().reset_index(drop=True)

    def compute_importance(i):
        nddt_trn = all_dates[i]
        nddt_tst = all_dates[i+12]
        X_trn, y_trn = X[X.index<=nddt_trn], y[y.index<=nddt_trn]
        X_tst, y_tst= X[(X.index>nddt_trn)&(X.index<=nddt_tst)], y[(y.index>nddt_trn)&(y.index<=nddt_tst)]

        mod = model.fit(X_trn, y_trn)
        y_pred = mod.predict(X_tst)
        R2_oos_all.append(R_oos(y_tst, y_pred))

        importance_dict = {"All Features": R2_oos_all}

        # Assess importance of each feature
        for feature in fts:
            X_trn_drop = X_trn.drop(feature, axis=1).copy()
            X_tst_drop = X_tst.drop(feature, axis=1).copy()

            mod_drop = LinearRegression().fit(X_trn_drop, y_trn)
            y_pred_drop = mod_drop.predict(X_tst_drop)

            R2_oos_drop = R_oos(y_tst, y_pred_drop)
            importance_dict[feature] = R2_oos_all - R2_oos_drop
            
            var_imp_dict[f"{features}"+f"{nddt_tst}"] = importance_dict
        print(f"{nddt_tst}Variable Importance Computation Complete")

    # Parallelize the for loop
    Parallel(n_jobs=-1)(delayed(compute_importance)(i) for i in np.arange(12*yrs_trn-1,len(all_dates[:-12]),12))

    return var_imp_dict